<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-обзор-данных" data-toc-modified-id="Загрузка-и-обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и обзор данных</a></span></li><li><span><a href="#Анализ-количества-книг" data-toc-modified-id="Анализ-количества-книг-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Анализ количества книг</a></span></li><li><span><a href="#Анализ-количества-обзоров-и-средней-оценки" data-toc-modified-id="Анализ-количества-обзоров-и-средней-оценки-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ количества обзоров и средней оценки</a></span></li><li><span><a href="#Анализ-самого-популярного-издательства" data-toc-modified-id="Анализ-самого-популярного-издательства-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Анализ самого популярного издательства</a></span></li><li><span><a href="#Анализ-самого-популярного-автора" data-toc-modified-id="Анализ-самого-популярного-автора-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Анализ самого популярного автора</a></span></li><li><span><a href="#Анализ-самых-активных-пользователей" data-toc-modified-id="Анализ-самых-активных-пользователей-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Анализ самых активных пользователей</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# SQL-проект по исследованию базы данных сервис для чтения книг

**Кейс:**

Компания купила крупный сервис для чтения книг по подписке. От руководства компании посупила задача — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

**Цели и задачи исследования:**

* Посчитать, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитать количество обзоров и среднюю оценку;
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, так мы исключим из анализа брошюры;
* Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
* Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [2]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

In [3]:
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [4]:
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

In [7]:
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

## Загрузка и обзор данных

In [8]:
# формируем запрос и выводим первые 5 строк таблицы books
query = '''SELECT * 
           FROM books 
           LIMIT 5'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [22]:
# выводим количество строк таблицы books
query = '''SELECT COUNT(book_id)
           FROM books'''
get_sql_data(query)

,count
0,1000


In [9]:
# формируем запрос и выводим первые 5 строк таблицы authors
query = '''SELECT * 
           FROM authors 
           LIMIT 5'''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [13]:
# выводим количество строк таблицы authors
query = '''SELECT COUNT(author_id)
           FROM authors'''
get_sql_data(query)

,count
0,636


In [10]:
# формируем запрос и выводим первые 5 строк таблицы publishers
query = '''SELECT * FROM publishers LIMIT 5'''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [14]:
# выводим количество строк таблицы publishers
query = '''SELECT COUNT(publisher_id)
           FROM publishers'''
get_sql_data(query)

,count
0,340


In [11]:
# формируем запрос и выводим первые 5 строк таблицы ratings
query = '''SELECT * FROM ratings LIMIT 5'''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [15]:
# выводим количество строк таблицы ratings
query = '''SELECT COUNT(rating_id)
           FROM ratings'''
get_sql_data(query)

,count
0,6456


In [18]:
# формируем запрос и выводим первые 5 строк таблицы reviews
query = '''SELECT * FROM reviews LIMIT 5'''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [21]:
# выводим количество строк таблицы reviews
query = '''SELECT COUNT(review_id)
           FROM reviews'''
get_sql_data(query)

,count
0,2793


У нас имеется 4 таблицы: 

Таблица `books` cодержит данные о книгах, имеет следующую структуру:  
* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.

В таблице 1000 строк.

Таблица `authors` cодержит данные об авторах, имеет следующую структуру:  
* `author_id` — идентификатор автора;
* `author` — имя автора.

В таблице 340 строк.

Таблица `publishers` cодержит данные об издательствах, имеет следующую структуру:  
* `publisher_id` — идентификатор издательства;
* `publisher` — название издательства;

В таблице 6456 строк.

Таблица `ratings` cодержит данные о пользовательских оценках книг, имеет следующую структуру:  
* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.

Таблица `reviews` cодержит данные о пользовательских обзорах на книги, имеет следующую структуру:
* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, написавшего обзор;
* `text` — текст обзора.

В таблице 2793 строк.

## Анализ количества книг

Посчитаем, сколько книг вышло после 1 января 2000 года.

In [88]:
# делаем срез данных по дате и считаем количество книг по колонке book_id 
query = '''SELECT COUNT(book_id) AS books_count
           FROM books
           WHERE publication_date > '2000-01-02' '''
get_sql_data(query)

,books_count
0,819


## Анализ количества обзоров и средней оценки

Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [39]:
# объединяем таблицы books, ratings и reviews по колонке book_id, 
# выводим поля с id книги, названием книги, считаем количество обзоров и средюю оценку для каждой книги,
# сортируем тоблицу сначала по убыванию среднего рейтина, затем по убыванию количества обзоров
query = '''SELECT b.book_id,
                  b.title,
                  COUNT(rv.review_id) AS review_count,
                  AVG(rt.rating) AS avg_raiting
           FROM books AS b
           LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
           LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
           GROUP BY b.book_id
           ORDER BY avg_raiting DESC, review_count DESC'''
get_sql_data(query)

,book_id,title,review_count,avg_raiting
0,17,A Dirty Job (Grim Reaper #1),16,5.00
1,553,School's Out—Forever (Maximum Ride #2),12,5.00
2,444,Moneyball: The Art of Winning an Unfair Game,9,5.00
3,347,In the Hand of the Goddess (Song of the Liones...,6,5.00
4,610,Tai-Pan (Asian Saga #2),4,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,12,2.25
996,202,Drowning Ruth,9,2.00
997,371,Junky,4,2.00
998,316,His Excellency: George Washington,4,2.00


## Анализ самого популярного издательства

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц, так мы исключим из анализа брошюры.

In [56]:
# объединяем таблицы books и publishers по колонке publisher_id, делаем срез по количеству страниц
# группируем по колонке publisher и выводим поля с издательством и считаем количество изданных книг у каждого
# издателя, сортируем по убыванию колонки books_count и оставляем только первые 5 строк
query = '''SELECT p.publisher,
                  COUNT (b.book_id) AS books_count
           FROM books AS b
           LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher
           ORDER BY books_count DESC 
           LIMIT 5'''
get_sql_data(query)

,publisher,books_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


## Анализ самого популярного автора

Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками.

In [69]:
# создаем общее табличное выражение rq_50, в котором посчитаем книги с 50 и более оценками, 
# отсортируем таблицу по количеству оценок raiting_count в порядке убывания
# затем создаем основную таблицу: присоединяем к таблице books таблицы authors, ratings и табличное выражение rq_50
# делаем сраз по колонке raiting_count от 50 и более, группируем данные по авторам
# выводим поля с author и считаем среднюю оценку, округлив ее до 2 цифр после запятой и оставляем только 1 строку
query = '''WITH rq_50 AS (SELECT b.book_id,
                                COUNT(rt.rating) AS raiting_count
                         FROM books AS b
                         LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
                         GROUP BY b.book_id
                         ORDER BY raiting_count DESC)
           
           SELECT a.author,
                  ROUND(AVG(rt.rating),2) AS avg_raiting
           FROM books AS b
           JOIN rq_50 as rq ON b.book_id = rq.book_id
           LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
           LEFT JOIN authors AS a ON b.author_id = a.author_id
           WHERE rq.raiting_count >= 50
           GROUP BY a.author
           ORDER BY avg_raiting DESC 
           LIMIT 1'''
get_sql_data(query)

,author,avg_raiting
0,J.K. Rowling/Mary GrandPré,4.29


## Анализ самых активных пользователей

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [89]:
# создаем табличное выражение rq_48 на основе таблицы ratings, в котором сгруппируем данные по пользователям
# и выведем поля с именами пользователей и количеством оценок rating_count, которые поставил каждой из них 
# и отсортируем данные по убыванию количества оценок 
# затем создадим второе табличное выражение users: соедимим тоблицу reviews и табличное выражение rq_48 по именам
# пользователей и сделаем срез по колонке rating_count больше 48, выведем поля username и посчитаем количество
# обзоров для каждого пользователя
# и затем посчитаем среднее число по колонке reviews_count
query = '''WITH rq_48 AS (SELECT rt.username,
                                 COUNT(rt.rating) AS rating_count
                          FROM ratings AS rt
                          GROUP BY rt.username
                          ORDER BY rating_count DESC),
           
                users AS (SELECT rq.username,
                                 COUNT(review_id) AS reviews_count
                          FROM reviews AS rv
                          INNER JOIN rq_48 as rq ON rv.username = rq.username
                          WHERE rq.rating_count > 48 
                          GROUP BY rq.username)
                          
SELECT AVG(reviews_count) AS avg_reviews
FROM users'''
get_sql_data(query)

,avg_reviews
0,24.0


## Выводы

* После 1 января 2000 года вышло ***819*** книг.  
* На первом месте по количеству обзоров и по средней оценке книга ***"A Dirty Job (Grim Reaper #1)"***. У нее было 16 обзоров, средний рейтинг составляет 5.00.  
* Издательство, которое выпустило наибольшее число книг, толще 50 страниц, ***Penguin Books***. В издательстве насчитывается 42 таких книги.  
* Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками) - ***J.K. Rowling/Mary GrandPré*** с рейтингом 4.29.  
* Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок, - ***24***.